In [14]:
import csv
import numpy as np

file_path = 'files/filtered_feature_table.csv'
data = []
with open(file_path, newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        del row[0]
        data.append(row)
del data[0]
for i in range(len(data)):
    for j in range(len(data[i])):
        if float(data[i][j]) > 0:
            data[i][j] = 1
        else:
            data[i][j] = 0

matrix_size = len(data)
mat = np.array(data)
matriz_adj = np.zeros((matrix_size, matrix_size)) #criando a matriz de adjacência com zeros

for i in range(len(data)):
    for j in range(i + 1, len(data)):
        # Verificando se duas bactérias estão presentes em mais de três pacientes
        if np.sum(mat[i] & mat[j]) >= 3:
            matriz_adj[i][j] = 1
            matriz_adj[j][i] = 1

N = {
    i: set(num for num, j in enumerate(row) if j)
    for i, row in enumerate(matriz_adj)
}

def BronKerbosch1(P, R=None, X=None):
    P = set(P) # 1
    R = set() if R is None else R # 1
    X = set() if X is None else X # 1
    if not P and not X: # 1
        yield R # 1
    while P:
        v = P.pop() # 1
        yield from BronKerbosch1(
            P=P.intersection(N[v]), R=R.union([v]), X=X.intersection(N[v])) # 3^n
        X.add(v) # 1

P = N.keys()
print(list(BronKerbosch1(P)))


KeyboardInterrupt: 

In [5]:
import csv
import numpy as np

file_path = 'files/feature_table.csv'
data = []
with open(file_path, newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        del row[0]
        data.append(row)
del data[0]
del data[100:]
for row in data:
    del row[0]

for i in range(len(data)):
    for j in range(len(data[i])):
        if float(data[i][j]) > 0:
            data[i][j] = 1
        else:
            data[i][j] = 0

matrix_size = len(data)
mat = np.array(data)
matriz_adj = np.zeros((matrix_size, matrix_size))

for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if np.sum(mat[i] & mat[j]) >= 3:
            matriz_adj[i][j] = 1
            matriz_adj[j][i] = 1

N = {
    i: set(num for num, j in enumerate(row) if j)
    for i, row in enumerate(matriz_adj)
}

def BronKerbosch1(P, R=None, X=None):
    P = set(P)
    R = set() if R is None else R
    X = set() if X is None else X
    if not P and not X:
        return [list(R)]
    cliques = []
    while P:
        v = P.pop()
        new_R = R.union([v])
        new_P = P.intersection(N[v])
        new_X = X.intersection(N[v])
        cliques.extend(BronKerbosch1(new_P, new_R, new_X))
        X.add(v)
    return cliques

P = N.keys()
cliques = list(BronKerbosch1(P))

# Salvando os cliques em um arquivo CSV
output_file_path = 'files/cliques_result.csv'
with open(output_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerows(cliques)

print(f"Os cliques foram armazenados em {output_file_path}")


Os cliques foram armazenados em files/cliques_result.csv


In [11]:
import csv
import numpy as np

def bron_kerbosch(graph, clique, candidates, excluded):
    if not candidates and not excluded:
        print("Clique encontrada:", clique)
        return

    for vertex in sorted(list(candidates), key=lambda v: len(graph[v]), reverse=True):
        neighbors = set(graph[vertex])
        bron_kerbosch(
            graph,
            clique + [vertex],
            candidates.intersection(neighbors),
            excluded.intersection(neighbors),
        )
        candidates.remove(vertex)
        excluded.add(vertex)

file_path = 'files/feature_table.csv'

# Leitura do arquivo CSV e criação da matriz de adjacência
data = []
with open(file_path, newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        del row[0]
        data.append(row)
del data[0]
for row in data:
    del row[0]

# Remoção das linhas que contêm menos de 5 valores diferentes de 0
filtered_data = [row for row in data if sum(1 for value in row[1:] if float(value) != 0) >= 5]

for i in range(len(data)):
    for j in range(len(data[i])):
        if float(data[i][j]) > 0:
            data[i][j] = 1
        else:
            data[i][j] = 0

matrix_size = len(data)
mat = np.array(data)
matriz_adj = np.zeros((matrix_size, matrix_size))

for i in range(len(data)):
    for j in range(i + 1, len(data)):
        if np.sum(mat[i] & mat[j]) >= 3:
            matriz_adj[i][j] = 1
            matriz_adj[j][i] = 1

# Convertendo a matriz de adjacência para um formato de dicionário de grafos
graph_dict = {i: np.nonzero(row)[0].tolist() for i, row in enumerate(matriz_adj)}

# Aplicação do algoritmo de Bron–Kerbosch com heurísticas
bron_kerbosch(graph_dict, [], set(graph_dict.keys()), set())


Clique encontrada: [0, 100, 107, 115, 119, 640, 641, 642, 643, 703, 647, 678, 421, 645, 692, 693, 687, 63, 105, 634, 644, 419, 1552, 62, 663, 61, 471, 675, 689, 665, 705, 1078, 1619, 101, 131, 629, 106, 1722, 639, 109, 674, 679, 464, 1484, 1549, 470, 681, 595, 690, 108, 103, 420, 673, 699, 666, 704, 444, 450, 1622, 118, 459, 1724, 706, 1705, 116, 662, 293, 443, 466, 93, 682, 110, 422, 448, 696, 1164, 1725, 701, 400, 440, 1666, 12, 1007, 311, 315, 412, 564, 111, 306, 338, 648, 112, 441, 1723, 442, 1620, 452, 636, 1719, 451, 294, 439, 465, 667, 292, 460, 462, 670, 290, 1623, 295, 592, 449, 406, 454, 301, 453, 1726, 734, 481, 700, 468, 120, 417, 463, 1547, 1551, 461, 455, 414, 676, 660, 1548, 661, 467, 1556, 423, 339, 303, 456, 1625, 1479, 505, 1706, 445, 98, 308, 1004, 407, 457, 309, 99, 122, 671, 332, 1005, 304, 683, 1553, 604, 489, 495, 121, 565, 1542, 1543, 566, 4, 305, 1060, 1087, 652, 334, 337, 416, 1008, 684, 299, 638, 698, 1165, 728, 694, 1720, 537, 1640, 1071, 1718, 733, 1475, 64

KeyboardInterrupt: 